Step 1: Load the Dataset

In [66]:
import pandas as pd

# Load your dataset
df = pd.read_csv('/content/sample_data/train.csv')

# Preview the first few rows
print(df.head())


   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Fuel_Type Transmission Owner_Type     Mileage   Engine  \
0              41000    Diesel       Manual      First  19.67 kmpl  1582 CC   
1              46000    Petrol       Manual      First    13 km/kg  1199 CC   
2              87000    Diesel       Manual      First  20.77 kmpl  1248 CC   
3              40670    Diesel    Automatic     Second   15.2 kmpl  1968 CC   
4              86999    Diesel       Manual      First  23.08 kmpl  1461 CC   

       Power  Seats  New_Price  Price  
0  126.2 bhp    5.0        NaN  12.50  
1   88.7 bhp    5.0  8.61 Lakh

Step 2: Handling Missing Values



In [67]:
# Check for missing values
print(df.isnull().sum())

# Impute missing values:
# For 'Mileage', 'Engine', and 'Power', use the mode of each column
df['Mileage'] = df['Mileage'].fillna(df['Mileage'].mode()[0])
df['Engine'] = df['Engine'].fillna(df['Engine'].mode()[0])
df['Power'] = df['Power'].fillna(df['Power'].mode()[0])

# For 'Seats', use the median as it's numerical
df['Seats'] = df['Seats'].fillna(df['Seats'].median())

# Drop 'New_Price' column due to high percentage of missing values
df = df.drop(columns=['New_Price'])

# Confirm there are no missing values left
print(df.isnull().sum())


Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64
Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64


Step 3: Remove Units from Numerical Columns

In [68]:
# Define a function to remove non-numeric characters
def extract_numeric(value):
    if isinstance(value, str):
        # Keep only digits and decimal points
        return float(''.join(filter(lambda x: x.isdigit() or x == '.', value)))
    return value

# Apply this function to the relevant columns
df['Mileage'] = df['Mileage'].apply(extract_numeric)
df['Engine'] = df['Engine'].apply(extract_numeric)
df['Power'] = df['Power'].apply(extract_numeric)

# Note: Since 'New_Price' was dropped earlier due to high missing values, skip it here.
# If it's still in your data, you can uncomment the following line:
# df['New_Price'] = df['New_Price'].apply(extract_numeric)

# Verify the changes
print(df[['Mileage', 'Engine', 'Power']].head())


   Mileage  Engine   Power
0    19.67  1582.0  126.20
1    13.00  1199.0   88.70
2    20.77  1248.0   88.76
3    15.20  1968.0  140.80
4    23.08  1461.0   63.10


Step 4: One-Hot Encoding for Categorical Variables

In [69]:
# Perform one-hot encoding for 'Fuel_Type' and 'Transmission'
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True, dtype=int)

# Verify the encoding and data types
print("\nFirst few rows:")
print(df.head())
print("\nData types of encoded columns:")
print(df.dtypes)


First few rows:
   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  Price  \
0              41000      First    19.67  1582.0  126.20    5.0  12.50   
1              46000      First    13.00  1199.0   88.70    5.0   4.50   
2              87000      First    20.77  1248.0   88.76    7.0   6.00   
3              40670     Second    15.20  1968.0  140.80    5.0  17.74   
4              86999      First    23.08  1461.0   63.10    5.0   3.50   

   Fuel_Type_Electric  Fuel_Type_Petrol  Transmission_Manual  
0                   0                 0                    1

Step 5: Add New Feature (Current Age of Car)

In [70]:
from datetime import datetime

# Calculate current year
current_year = datetime.now().year

# Create a new column 'Car_Age'
df['Car_Age'] = current_year - df['Year']

# Verify the new feature
print(df[['Year', 'Car_Age']].head())


   Year  Car_Age
0  2015        9
1  2011       13
2  2012       12
3  2013       11
4  2013       11


Step 6: Data Manipulation Operations

In [71]:
# Select specific columns (e.g., 'Name', 'Location', 'Year', and 'Price')
selected_df = df[['Name', 'Location', 'Year', 'Price']]
print(selected_df.head())

# Filter rows (e.g., cars priced over 10 lakhs)
filtered_df = df[df['Price'] > 10]
print(filtered_df.head())

# Rename columns (e.g., rename 'Kilometers_Driven' to 'Km_Driven')
df = df.rename(columns={'Kilometers_Driven': 'Km_Driven'})
print(df.columns)

# Mutate
from datetime import datetime
# Calculate the current year
current_year = datetime.now().year
# Add a new column 'Car_Age' representing the age of the car
df['Car_Age'] = current_year - df['Year']
print(df[['Year', 'Car_Age']].head())

# Arrange (sort) by price in descending order
sorted_df = df.sort_values(by='Price', ascending=False)
print(sorted_df.head())

# Summarize with Group By
# Group by 'Location' and calculate the mean price for each location
location_summary = df.groupby('Location')['Price'].mean().reset_index()
# Rename the summarized column for clarity
location_summary = location_summary.rename(columns={'Price': 'Average_Price'})
print(location_summary.head())



                               Name    Location  Year  Price
0  Hyundai Creta 1.6 CRDi SX Option        Pune  2015  12.50
1                      Honda Jazz V     Chennai  2011   4.50
2                 Maruti Ertiga VDI     Chennai  2012   6.00
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013  17.74
4            Nissan Micra Diesel XV      Jaipur  2013   3.50
    Unnamed: 0                               Name    Location  Year  \
0            1   Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
3            4    Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
5            7  Toyota Innova Crysta 2.8 GX AT 8S      Mumbai  2016   
11          13   Land Rover Range Rover 2.2L Pure       Delhi  2014   
12          14     Land Rover Freelander 2 TD4 SE        Pune  2012   

    Kilometers_Driven Owner_Type  Mileage  Engine  Power  Seats  Price  \
0               41000      First    19.67  1582.0  126.2    5.0  12.50   
3               40670     Second    15.20  1968.0  140.8   